In [1]:
 !unzip UCI_HAR_Dataset.zip

Archive:  UCI_HAR_Dataset.zip
  inflating: UCI_HAR_Dataset/.DS_Store  
  inflating: UCI_HAR_Dataset/_DS_Store  
  inflating: UCI_HAR_Dataset/activity_labels.txt  
   creating: UCI_HAR_Dataset/csv_files/
  inflating: UCI_HAR_Dataset/csv_files/test.csv  
  inflating: UCI_HAR_Dataset/csv_files/train.csv  
  inflating: UCI_HAR_Dataset/features.txt  
  inflating: UCI_HAR_Dataset/features_info.txt  
  inflating: UCI_HAR_Dataset/README.txt  
   creating: UCI_HAR_Dataset/test/
   creating: UCI_HAR_Dataset/test/Inertial Signals/
  inflating: UCI_HAR_Dataset/test/Inertial Signals/body_acc_x_test.txt  
  inflating: UCI_HAR_Dataset/test/Inertial Signals/body_acc_y_test.txt  
  inflating: UCI_HAR_Dataset/test/Inertial Signals/body_acc_z_test.txt  
  inflating: UCI_HAR_Dataset/test/Inertial Signals/body_gyro_x_test.txt  
  inflating: UCI_HAR_Dataset/test/Inertial Signals/body_gyro_y_test.txt  
  inflating: UCI_HAR_Dataset/test/Inertial Signals/body_gyro_z_test.txt  
  inflating: UCI_HAR_Dataset/test

In [0]:
### Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import random
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)


# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras import backend as K

from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout,Conv2D,MaxPooling2D

Using TensorFlow backend.


In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}


ACTIVITIES1 = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
}

ACTIVITIES2 = {
    0: 'SITTING',
    1: 'STANDING',
    2: 'LAYING',
}

ACTIVITIES3 = {
    0: 'SITTING',
    1: 'STANDING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

# Utility function to print the confusion matrix
def confusion_matrix_1(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES1[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES1[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])
  
  
# Utility function to print the confusion matrix
def confusion_matrix_2(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES2[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES2[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])
  
# Utility function to print the confusion matrix
def confusion_matrix_3(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES3[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES3[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data Gathering

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:
def load_y(subset):
    global dir_path
    if subset is "train":
        y_path = DATADIR + '/train/y_train.txt'
    elif subset is "test":
        y_path = DATADIR + '/test/y_test.txt'

    with open(y_path) as f:
        container = f.readlines()

    result = []
    for line in container:
        num_str = line.strip()
        result.append(int(num_str))
    return np.array(result)

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# Import Keras
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [0]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [14]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
#n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [0]:
unique, counts = np.unique(Y_test, return_counts=True)

In [16]:
print("Unique classes and their counts--")
print(unique)
print(counts)

Unique classes and their counts--
[1 2 3 4 5 6]
[496 471 420 491 532 537]


### Loading Data

In [0]:
# Load all train and test data (* dynamic and static data are mixed.)

X_train_all = load_signals("train")   # at this stage, the data includes both dynamic and static HAR data
Y_train_all = load_y("train")

In [0]:
X_test_all = load_signals("test")
Y_test_all = load_y("test")

In [19]:
print(X_train_all.shape, Y_test_all.shape)
print(X_test_all.shape, Y_test_all.shape) #[samples, timesteps, features]

(7352, 128, 9) (2947,)
(2947, 128, 9) (2947,)


### Separating Data- Dynamic (WALKING, WALKING UPSTAIRS,  WALKING DOWNSTAIRS)

In [0]:
import random

# Select dynamic HAR train data

dynamic_1 = np.where(Y_train_all == 1)[0]
dynamic_2 = np.where(Y_train_all == 2)[0]
dynamic_3 = np.where(Y_train_all == 3)[0]
dynamic = np.concatenate([dynamic_1, dynamic_2, dynamic_3])
dynamic_list = dynamic.tolist()

# Shuffle dynamic data index
r = random.random()
random.shuffle(dynamic_list, lambda: r)

dynamic = np.array(dynamic_list)

X_train = X_train_all[dynamic]
Y_train = Y_train_all[dynamic]

In [0]:
print ("train_dynamic shape: ", Y_train.shape)

np.unique(Y_train)

train_dynamic shape:  (3285,)


array([1, 2, 3])

In [0]:
X_train.shape

(3285, 128, 9)

In [0]:
dynamic_1 = np.where(Y_test_all == 1)[0]
dynamic_2 = np.where(Y_test_all == 2)[0]
dynamic_3 = np.where(Y_test_all == 3)[0]
dynamic = np.concatenate([dynamic_1, dynamic_2, dynamic_3])

X_test = X_test_all[dynamic]
Y_test = Y_test_all[dynamic]

In [0]:
print ("test_dynamic shape: ", Y_test.shape)

np.unique(Y_test)

test_dynamic shape:  (1387,)


array([1, 2, 3])

In [0]:
# Convert to one hot encoding vector

from keras.utils import np_utils

Y_train_dynamic_ohe=np.asarray(pd.get_dummies(Y_train))
Y_test_dynamic_ohe=np.asarray(pd.get_dummies(Y_test))


In [0]:
Y_train_dynamic_ohe

array([[1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

In [0]:
Y_test_dynamic_ohe

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

### Building Model for Dynamic Data

In [0]:
X_train.shape

(3285, 128, 9)

In [73]:
model1 = Sequential()

model1.add(Conv1D(32, 3, input_shape=(128, 9), activation='relu',kernel_initializer = 'he_normal'))
model1.add(MaxPooling1D(2))
model1.add(Dropout(0.2))
model1.add(BatchNormalization())


model1.add(Conv1D(64, 3, activation='relu',kernel_initializer = 'he_normal'))
model1.add(MaxPooling1D(2))
model1.add(BatchNormalization())
#model1.add(Dropout(0.2))

model1.add(Conv1D(80, 3,  activation='relu',kernel_initializer = 'he_normal'))
model1.add(MaxPooling1D(2))
model1.add(Dropout(0.2))
model1.add(BatchNormalization())

model1.add(Flatten())

model1.add(Dense(3, activation='softmax'))


model1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer = 'adam')

model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_29 (Conv1D)           (None, 126, 32)           896       
_________________________________________________________________
max_pooling1d_26 (MaxPooling (None, 63, 32)            0         
_________________________________________________________________
dropout_44 (Dropout)         (None, 63, 32)            0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 63, 32)            128       
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 61, 64)            6208      
_________________________________________________________________
max_pooling1d_27 (MaxPooling (None, 30, 64)            0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 30, 64)            256       
__________

In [74]:
model1.fit(X_train, Y_train_dynamic_ohe,batch_size=32, epochs=50, verbose=2, validation_data=(X_test, Y_test_dynamic_ohe))


Train on 3285 samples, validate on 1387 samples
Epoch 1/50
 - 7s - loss: 0.7209 - acc: 0.7081 - val_loss: 0.5022 - val_acc: 0.8198
Epoch 2/50
 - 1s - loss: 0.1686 - acc: 0.9385 - val_loss: 0.3290 - val_acc: 0.8666
Epoch 3/50
 - 1s - loss: 0.0781 - acc: 0.9723 - val_loss: 0.3665 - val_acc: 0.8609
Epoch 4/50
 - 1s - loss: 0.0442 - acc: 0.9896 - val_loss: 0.4038 - val_acc: 0.8335
Epoch 5/50
 - 1s - loss: 0.0312 - acc: 0.9903 - val_loss: 0.3472 - val_acc: 0.8645
Epoch 6/50
 - 1s - loss: 0.0270 - acc: 0.9909 - val_loss: 0.2836 - val_acc: 0.8904
Epoch 7/50
 - 1s - loss: 0.0204 - acc: 0.9927 - val_loss: 0.1611 - val_acc: 0.9373
Epoch 8/50
 - 1s - loss: 0.0176 - acc: 0.9954 - val_loss: 0.1315 - val_acc: 0.9459
Epoch 9/50
 - 1s - loss: 0.0174 - acc: 0.9936 - val_loss: 0.1859 - val_acc: 0.9214
Epoch 10/50
 - 1s - loss: 0.0080 - acc: 0.9976 - val_loss: 0.1842 - val_acc: 0.9301
Epoch 11/50
 - 1s - loss: 0.0053 - acc: 0.9985 - val_loss: 0.0990 - val_acc: 0.9582
Epoch 12/50
 - 1s - loss: 0.0106 - ac

In [75]:
# Confusion Matrix
print(confusion_matrix(Y_test_dynamic_ohe, model1.predict(X_test)))

Pred                WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                                             
WALKING                 482                  10                 4
WALKING_DOWNSTAIRS        0                 409                11
WALKING_UPSTAIRS          0                   2               469


In [76]:
score1 = model1.evaluate(X_test, Y_test_dynamic_ohe)

1387/1387 [==============================] - 0s 152us/step


In [77]:
score1

[0.043732418487859935, 0.9805335255948089]

### Separating Data- Static (SITTING, STANDING,  LAYING)

In [0]:
import random

# Select static HAR train data

static_1 = np.where(Y_train_all == 4)[0]
static_2 = np.where(Y_train_all == 5)[0]
static_3 = np.where(Y_train_all == 6)[0]
static = np.concatenate([static_1, static_2, static_3])
static_list = static.tolist()

# Shuffle static data index
r = random.random()
random.shuffle(static_list, lambda: r)

static = np.array(static_list)

X_train = X_train_all[static]
Y_train = Y_train_all[static]

In [21]:
print ("train_static shape: ", Y_train.shape)

np.unique(Y_train)

train_static shape:  (4067,)


array([4, 5, 6])

In [0]:
X_train.shape

(4067, 128, 9)

In [0]:
# Select static HAR test data


static_1 = np.where(Y_test_all == 4)[0]
static_2 = np.where(Y_test_all == 5)[0]
static_3 = np.where(Y_test_all == 6)[0]
static = np.concatenate([static_1, static_2, static_3])

X_test = X_test_all[static]
Y_test = Y_test_all[static]

In [23]:
print ("static shape: ", Y_test.shape)

np.unique(Y_test)

static shape:  (1560,)


array([4, 5, 6])

In [0]:
# Convert to one hot encoding vector

from keras.utils import np_utils

Y_train_static_ohe=np.asarray(pd.get_dummies(Y_train))
Y_test_static_ohe=np.asarray(pd.get_dummies(Y_test))


In [25]:
Y_train_static_ohe

array([[1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0]], dtype=uint8)

In [26]:
Y_test_static_ohe

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

### Building Model for Static Data

In [28]:
X_train.shape

(4067, 128, 9)

In [56]:
model2 = Sequential()


model2.add(Conv1D(50, 3, input_shape=(128, 9), activation='tanh',kernel_initializer = 'he_normal'))
model2.add(MaxPooling1D(2))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))


model2.add(Conv1D(100, 3, activation='tanh',kernel_initializer = 'he_normal'))
model2.add(MaxPooling1D(2))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))



model2.add(LSTM(32,activation = 'tanh',kernel_initializer = 'he_normal',return_sequences = True))
model2.add(Dropout(0.5))


model2.add(LSTM(80,activation = 'tanh',kernel_initializer = 'he_normal',return_sequences = True))
model2.add(Dropout(0.5))



model2.add(Flatten())


model2.add(Dense(3, activation='softmax'))



model2.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer = 'adam')

model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_20 (Conv1D)           (None, 126, 50)           1400      
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 63, 50)            0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 63, 50)            200       
_________________________________________________________________
dropout_34 (Dropout)         (None, 63, 50)            0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 61, 100)           15100     
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 30, 100)           0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 30, 100)           400       
__________

In [57]:
model2.fit(X_train, Y_train_static_ohe,batch_size=32, epochs=161, verbose=2, validation_data=(X_test, Y_test_static_ohe))


Train on 4067 samples, validate on 1560 samples
Epoch 1/161
 - 25s - loss: 0.2918 - acc: 0.8812 - val_loss: 0.4686 - val_acc: 0.8481
Epoch 2/161
 - 19s - loss: 0.2519 - acc: 0.8992 - val_loss: 0.2824 - val_acc: 0.9038
Epoch 3/161
 - 19s - loss: 0.2474 - acc: 0.8962 - val_loss: 0.3019 - val_acc: 0.8808
Epoch 4/161
 - 19s - loss: 0.2167 - acc: 0.9134 - val_loss: 0.2632 - val_acc: 0.8776
Epoch 5/161
 - 19s - loss: 0.2258 - acc: 0.9031 - val_loss: 0.3321 - val_acc: 0.8795
Epoch 6/161
 - 19s - loss: 0.2202 - acc: 0.9073 - val_loss: 0.2725 - val_acc: 0.9064
Epoch 7/161
 - 19s - loss: 0.2049 - acc: 0.9120 - val_loss: 0.3297 - val_acc: 0.8814
Epoch 8/161
 - 19s - loss: 0.2048 - acc: 0.9127 - val_loss: 0.3491 - val_acc: 0.8583
Epoch 9/161
 - 19s - loss: 0.2026 - acc: 0.9127 - val_loss: 0.2849 - val_acc: 0.9103
Epoch 10/161
 - 19s - loss: 0.1838 - acc: 0.9270 - val_loss: 0.2996 - val_acc: 0.8987
Epoch 11/161
 - 19s - loss: 0.1873 - acc: 0.9208 - val_loss: 0.3197 - val_acc: 0.8904
Epoch 12/161
 -

In [58]:
# Confusion Matrix
print(confusion_matrix_2(Y_test_static_ohe, model2.predict(X_test)))

Pred      LAYING  SITTING  STANDING
True                               
LAYING       537        0         0
SITTING        2      450        39
STANDING       0       22       510


In [61]:
score2 = model2.evaluate(X_test, Y_test_static_ohe)

1560/1560 [==============================] - 2s 2ms/step


In [64]:
score2

[0.18009876538986808, 0.9679153846153846]

### CONCLUSION

In [78]:
#http://zetcode.com/python/prettytable/

from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["MODEL CLASSES","ACCURACY"]

x.add_row(["MODEL 1- DYNAMIC(WALKING, DOWNWARDS, UPWADRS)",score1[1]])
x.add_row(["Model 2- STATIC(SITTING, STANDING, LAYING)",score2[1]])


print(x)


+-----------------------------------------------+--------------------+
|                 MODEL CLASSES                 |      ACCURACY      |
+-----------------------------------------------+--------------------+
| MODEL 1- DYNAMIC(WALKING, DOWNWARDS, UPWADRS) | 0.9805335255948089 |
|   Model 2- STATIC(SITTING, STANDING, LAYING)  | 0.9679153846153846 |
+-----------------------------------------------+--------------------+
